In [1]:
import os
import cv2

In [25]:
# Define source and destination root directories
source_root = 'pre_process_data'
dest_root = 'edge_images'
target_size = (244, 244)

In [26]:
# Create destination root directory if it doesn't exist
os.makedirs(dest_root, exist_ok=True)

In [27]:
# Walk through the source directory
for class_name in os.listdir(source_root):
    class_path = os.path.join(source_root, class_name)
    
    if os.path.isdir(class_path):
        # Create corresponding class folder in destination
        dest_class_path = os.path.join(dest_root, class_name)
        os.makedirs(dest_class_path, exist_ok=True)

        # Process each image in the class folder
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            dest_image_path = os.path.join(dest_class_path, image_name)

            # Read the image in grayscale
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                print(f"Skipping invalid image: {image_path}")
                continue

            # Resize the image
            resized_image = cv2.resize(image, target_size)

            # Apply Canny edge detection
            edges = cv2.Canny(resized_image, 100, 200)

            # Save the result
            cv2.imwrite(dest_image_path, edges)

print("Edge detection completed and resized images saved.")

Skipping invalid image: pre_process_data\CROW\.ipynb_checkpoints
Edge detection completed and resized images saved.


# CNN

In [28]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models

In [29]:
# Constants
IMAGE_SIZE = (244, 244)
BATCH_SIZE = 32
DATASET_PATH = 'edge_images'

In [30]:
# Load the dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    validation_split=0.2,
    subset="training",
    seed=123
)

Found 719 files belonging to 9 classes.
Using 576 files for training.


In [31]:
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    validation_split=0.2,
    subset="validation",
    seed=123
)

Found 719 files belonging to 9 classes.
Using 143 files for validation.


In [32]:
# Get number of classes
class_names = train_dataset.class_names
num_classes = len(class_names)

In [33]:
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [34]:
# ✅ Enhanced CNN Model with Dropout and Extra Layers
model = models.Sequential([
    layers.InputLayer(input_shape=(244, 244, 1)),

    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(2),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(2),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Flatten(),

    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),

    layers.Dense(num_classes, activation='softmax')
])

In [35]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 244, 244, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 122, 122, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 122, 122, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 122, 122, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 61, 61, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 61, 61, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 30, 30, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │      29,491,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 9)                   │           1,161 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,618,953 (112.99 MB)

 Trainable params: 29,618,569 (112.99 MB)

 Non-trainable params: 384 (1.50 KB)

In [38]:
# Train the model
hostory = model.fit(train_dataset, validation_data=val_dataset, epochs=50)

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.6177 - loss: 3.8791 - val_accuracy: 0.3846 - val_loss: 7.1391
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.7141 - loss: 2.1691 - val_accuracy: 0.5035 - val_loss: 4.3780
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.7247 - loss: 1.6512 - val_accuracy: 0.5035 - val_loss: 4.2575
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.7163 - loss: 1.8668 - val_accuracy: 0.5664 - val_loss: 2.4968
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.7523 - loss: 1.6727 - val_accuracy: 0.6224 - val_loss: 1.3453
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.7822 - loss: 1.2010 - val_accuracy: 0.6294 - val_loss: 1.1923
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.8110 - loss: 1.0900 - val_accuracy: 0.6713 - val_loss: 1.0068
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.8458 - loss: 0.9083 - val_accuracy: 0.6014 - val_loss:

In [47]:
# Save model
model.save('edge_classifier_improved.h5')

In [50]:
# This is correct for models trained on grayscale images
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'edge_images',
    image_size=(244, 244),
    batch_size=32,
    color_mode='grayscale',  # ✅ Keep grayscale
    shuffle=False
)


Found 719 files belonging to 9 classes.


In [51]:
# Load your trained model (if not already in memory)
model = tf.keras.models.load_model('edge_classifier_improved.h5')

# Evaluate
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")


23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 215ms/step - accuracy: 0.9488 - loss: 0.1863
Test Loss: 0.3169
Test Accuracy: 90.54%
